In [1]:
%run '02-01_Irreducible_Equivariant_Vector_Bundle_Over_Irreducible_Homogeneous_Variety.ipynb'

class Equivariant_Vector_Bundle_Over_Irreducible_Homogeneous_Variety (object) :

    def __add__ ( self , other ) :
        """Returns the sum ``self`` with ``other``."""
        assert type( other ) == self.__class__ , 'The input for ``other`` must be an object of the class ' + str( self.__class__ ) + '.'
        Sum = self.__class__( self.Summands()+other.Summands() )
        Sum.Tidy_Up()
        return Sum


    def __call__ ( self , *Twists ) :
        """Returns the tensor product of ``self`` with X.calO( Twists )."""
        assert len( Twists ) == len( self.Base_Space().Excluded_Nodes() ) , 'The length of ``Twists`` must coincide with the number of excluded nodes,' \
                                                                           + ' namley ' + str(len( self.Base_Space().Excluded_Nodes() )) + '.'
        return self * self.Base_Space().calO( *Twists )


    def __getitem__ ( self , Input ) :
        """Returns a single summand or slices of summands."""
        if Input in Integers() :
            Index = Input % len(self)
            return self.__class__( [ self.Summands(Index) ] )
        elif type(Input) == slice :
            Slice = Input
            return self.__class__( self.Summands(Slice) )
        else :
            raise ValueError('The input data is unknown.')


    def __init__ ( self , Summands=[] ) :
        """
        Initialize a irreducible equivariant vector bundle over an irreducible homogeneous variety.

        INPUT:
        - ``Base_Space`` -- Irreducible_Homogeneous_Variety;
        - ``Summands`` -- list of 2-tuple of the form ( <WeylCharacter> , <Irreducible_Equivariant_Vector_Bundle_Over_Irreducible_Homogeneous_Variety> ) (default:None);

        OUTPUT: None.
        """
        self._Summands = []

        if   type(Summands) == tuple and len(Summands) == 2 : Summands = [ Summands ]
        elif type(Summands) == Irreducible_Equivariant_Vector_Bundle_Over_Irreducible_Homogeneous_Variety : Summands = [ Summands ]

        if type(Summands) == list :
            if len(Summands) == 0 : raise ValueError('The input for ``Summands`` is empty.')

            for Summand_Counter , Summand in enumerate( Summands , start=1 ) :
                if type(Summand) == Irreducible_Equivariant_Vector_Bundle_Over_Irreducible_Homogeneous_Variety :
                    WCR = Summand.Base_Space().Parent_Group().Weyl_Character_Ring()
                    Summand = ( Summand , WCR(1) )

                try : Vector_Bundle , Multiplicity = Summand
                except : raise ValueError('The input data of the '+str(Summand_Counter)+'-th summand is inappropriate. It need to be a 2-tuple.')

                assert type(Vector_Bundle) == Irreducible_Equivariant_Vector_Bundle_Over_Irreducible_Homogeneous_Variety , \
                       'The input for ``Vector_Bundle`` at the '+str(Summand_Counter)+'-th summand need to be an irreducible vector bundle over an irreducible homogeneous variety.'

                if Summand_Counter == 1 : self._Base_Space = Vector_Bundle.Base_Space()
                else : assert Vector_Bundle.Base_Space() == self._Base_Space , \
                              'The vector bundle of the '+str(Summand_Counter)+'-th summand is not defined over the same base space as the vector bundle of the first summand.'

                WCR = self._Base_Space.Parent_Group().Weyl_Character_Ring()
                assert type(Multiplicity) == type(WCR(0)) , 'The multiplicity need to be a Weyl character.'

                if Multiplicity == 0 : Summand = self._Base_Space.VB(0)

                self._Summands += [ ( Vector_Bundle , Multiplicity ) ]

        else :
            raise ValueError('The input for ``Summands`` is inappropriate.')


    def __iter__ ( self ) :
        """Returns a generator for summands of ``self``."""
        for Summand_Index in [ 0 .. len(self)-1 ] :
            yield self[Summand_Index]

    def __len__ ( self ) :
        """Returns the number of summands of ``self``."""
        return len( self.Summands() )


    def __ne__ ( self , other ) :
        """Tests if two objects of the class ``Irreducible_Equivariant_Vector_Bundle_Over_Irreducible_Homogeneous_Variety`` do not coincide."""
        return not self == other


    def __repr__ ( self ) :
        """Returns all attributes which are necessary to initialize the object."""
        return self.Summands()


    def __mul__ ( self , other ) :
        """Returns the tensor product of ``self`` with ``other``."""
        assert type( other ) == self.__class__ , 'The input for ``other`` must be an object of the class ' + str( self.__class__ ) + '.'

        WCR = self._Base_Space.Parent_Group().Weyl_Character_Ring()
        Summands = []
        for Vector_Bundle_1 , Multiplicity_1 in self.Summands() :
            for Vector_Bundle_2 , Multiplicity_2 in other.Summands() :
                for Weyl_Character_HW , Weyl_Character_Multiplicity in Multiplicity_1 * Multiplicity_2 :
                    for Vector_Bundle , Vector_Bundle_Multiplicity in Vector_Bundle_1 * Vector_Bundle_2 :
                        Summands += [ ( Vector_Bundle , Vector_Bundle_Multiplicity * Weyl_Character_Multiplicity * WCR(Weyl_Character_HW) ) ]

        if len(Summands) == 0 : Product = self.Base_Space().calO().Multiply_By(0)
        else: Product = self.__class__( Summands )
        Product.Tidy_Up()
        return Product


    def __pow__ ( self , e ) :
        """Returns the power of ``self`` with respect to ``Exponent``."""
        assert e in Integers() , 'The exponent must be an positive integer.'
        Product = self.Base_Space().calO()
        if   e <  0 : Factor = self.Dual()
        elif e == 0 : return Product
        else : Factor = self
        for Counter in [ 1 .. abs(e) ] : Product *= Factor
        return Product
    # Synonyms for the function ``__pow__``
    def Tensor_Power ( self , e ) :
        """Returns the tensor power of ``self`` (cf. to the method ``__pow__``)"""
        return self^e
    def T ( self , e ) :
        """Returns the tensor power of ``self`` (cf. to the method ``__pow__``)"""
        return self^e


    def __str__ ( self ) :
        """Returns a one-line string as short description."""
        WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
        Strings = []
        for Vector_Bundle , Multiplicity in self.Summands() :
            if   Multiplicity == WCR(0) : Strings += [ '0' ]
            elif Multiplicity == WCR(1) : Strings += [ str(Vector_Bundle) ]
            else :
                if len(Multiplicity) == 1 : Strings += [ str(Multiplicity)+' * '+str(Vector_Bundle) ]
                else : Strings += [ '( '+str(Multiplicity)+' ) * '+str(Vector_Bundle) ]
        return ' + '.join(Strings)


    def Base_Space ( self ) :
        """Returns the attribute ``_Base_Space``."""
        return self._Base_Space


    def Cohomology ( self , *p ) :
        """Returns the cohomology."""
        Cohomology = {}
        for Summand , Multiplicity in self.Summands() :
            Result = Summand.Cohomology()
            if not Result == None :
                Degree , CohomologyGroup = Result
                if not Degree in Cohomology.keys() : Cohomology.update({ Degree : 0 })
                Cohomology[Degree] += Multiplicity*CohomologyGroup

        if   len(p) == 0 : return Cohomology
        elif len(p) == 1 :
            p = list(p)[0]
            if p in Cohomology.keys() : return Cohomology[p]
            else : return 0
        else :
            raise ValueError('The input for ``p`` is inappropriate.')
    # Synonym for the function ``Cohomology``
    def H ( self , *p ) :
        """Returns the cohomology."""
        return self.Cohomology( *p )


    def Components ( self ) :
        """Returns the irreducible components of ``self``."""

        for Vector_Bundle , Weyl_Character in self.Summands() :
            Components_Of_Current_Summand = self.Base_Space().calO().Multiply_By(0)
            Factor = self.__class__( Vector_Bundle )

            Dict = {}
            for Weight , Multiplicity in Weyl_Character.weight_multiplicities().items() :
                Weight = Weight.to_weight_space()
                if self.Base_Space().Is_Levi_Dominant( Weight ) :
                    rk = self.Base_Space().calU( Weight ).Rank()
                    if not rk in Dict.keys() : Dict.update({ rk : [] })
                    Dict[rk] += Multiplicity * [ Weight ]

            Rks = sorted(Dict.keys())
            Rks.reverse()

            for rk_1 in Rks :
                while 0 < len(Dict[rk_1]) :
                    Highest_Weight = Dict[rk_1][0]
                    Irreducible_Object = Irreducible_Equivariant_Vector_Bundle_Over_Irreducible_Homogeneous_Variety( self.Base_Space() , Highest_Weight=Highest_Weight )
                    Components_Of_Current_Summand += self.__class__( Irreducible_Object ) * Factor

                    for Weight , Multiplicity in Irreducible_Object.Weights() :
                        if self.Base_Space().Is_Levi_Dominant( Weight ) :
                            for Counter in [ 1 .. Multiplicity ] :
                                rk_2 = self.Base_Space().calU( Weight ).Rank()
                                Dict[rk_2].remove(Weight)

            yield Components_Of_Current_Summand


    def Degree ( self ) :
        """Returns the degree of ``self``."""
        Vector_Bundle , Multiplicity = self.Determinant_Bundle().Summands(0)
        Coefficients = Vector_Bundle.Highest_Weight().coefficients()
        if   len(Coefficients) == 0 : return 0
        elif len(Coefficients) == 1 : return Coefficients[0]
        else : return Coefficients


    def Determinant_Bundle ( self ) :
        """Returns the determinant line bundle of self (= rk-th top exterior power of ``self``)."""
        return self.Exterior_Power( self.Rank() )
    # Synonym for the function ``Determinant_Bundle``.
    def Det ( self ) :
        return self.Determinant_Bundle()


    def Euler_Sum ( self , other ) :
        """Returns sum_{ p } (-1)^p EXT^p(``self``,``other``)."""
        assert type( other ) == self.__class__ , 'The input for ``other`` must be an object of the class ' + str( self.__class__ ) + '.'
        return sum([ (-1)^Degree * Space for Degree , Space in self.EXT(other).items() ])


    def Dual ( self ) :
        """Returns the dual of ``self``."""
        return self.__class__( [ ( Summand.Dual() , Multiplicity ) for Summand , Multiplicity in self.Summands() ] )


    def EXT ( self , other , *p ) :
        """Returns EXT(^p)(``self``,``other``)."""
        assert type( other ) == self.__class__ , 'The input for ``other`` must be an object of the class ' + str( self.__class__ ) + '.'
        return ( self.Dual() * other ).Cohomology( *p )


    def Exterior_Power ( self , p ) :
        """
        Returns the ``e``-th exterior power of ``self``.

        We need to compute wedge^p( rmV^{ omega_1 } * calU^{ omega_2 } ) for a single summand.
        It holds
            wedge^p( rmV^{ omega_1 } * calU^{ omega_2 } )
                = alpha_p ( E_1 , ... , E_p  , F_1 , ... , F_p )
        with
            E_i = wedge^i rmV^{ omega_1 } * calO
        and
            F_j = wedge^j calU^{ omega_2 }.
        \alpha_p is the polynomial in 2p indeterminates defined by the formula
            sum_p alpha_p * T^p
                = prod_{ i,j >= 1 } ( 1 + X_i Y_j T ) =: beta
        where E_i is an elementary symmetric polynomial in the X_i's and F_j is such one in the Y_j's.

        https://concretenonsense.wordpress.com/2009/07/23/lambda-rings/ (Accessed:2023-02-08)
        """
        assert p in Integers() , 'The exponent ``e`` must be an integer.'

        if   p == 0 : return self.Base_Space().calO()
        elif p == 1 : return self
        elif p in [ 2 .. self.Rank() ] :

            #A = PolynomialRing( ZZ  , [ 'E'+str(Counter) for Counter in [ 1 .. p ] ] + [ 'F'+str(Counter) for Counter in [ 1 .. p ] ] )
            #E = A.gens()[ : p ]
            #F = A.gens()[ p : ]

            B_X = PolynomialRing( ZZ  , [ 'X'+str(Counter) for Counter in [ 1 .. p ] ] ) ; X = B_X.gens()
            B_Y = PolynomialRing( B_X , [ 'Y'+str(Counter) for Counter in [ 1 .. p ] ] ) ; Y = B_Y.gens()    ; B0 = B_Y
            B   = PolynomialRing( B0  , [ 'T' ] )                                        ; T = B.gens()[ 0 ]

            e = SymmetricFunctions(ZZ).elementary()
            f = SymmetricFunctions(B_X).elementary()

            if p == 0 : beta = [ 1 ]
            else : beta = prod([ 1 + X[Counter1-1]*Y[Counter2-1] * T for Counter1 in [ 1 .. p ] for Counter2 in [ 1 .. p ] ]).coefficients()

            WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()

            def Subroutine_For_Exterior_Power_Of_Summand ( Subexponent_For_Summand , Summand ) :
                Vector_Bundle , Multiplicity = Summand
                if   Vector_Bundle.Is_Trivial() :
                    return self.__class__( ( Vector_Bundle , Multiplicity.exterior_power(Subexponent_For_Summand) ) )
                elif Multiplicity == WCR(1) :
                    Summand = self.Base_Space().calO().Multiply_By(0)
                    Summand += sum([ self.__class__( ( S_VB , S_Mult*WCR(1) ) ) for S_VB , S_Mult in Vector_Bundle.Exterior_Power(Subexponent_For_Summand) ])
                    return Summand
                else :
                    alpha = self.Base_Space().calO().Multiply_By(0)

                    if   Subexponent_For_Summand == 0 : return self.Base_Space().calO()
                    elif Subexponent_For_Summand == 1 : return self.__class__( Summand )

                    for Degree1 , Coefficient1 in f.from_polynomial( beta[Subexponent_For_Summand] ).monomial_coefficients().items() :

                        #Factor1 = prod([ F[Counter-1] for Counter in Degree1 ])
                        Factor_VB = self.Base_Space().calO()
                        for Counter in Degree1 :
                            # Since the method ``Exterior_Power`` of Vector_Bundle works only as generat we need to define the cooresponding "elementary symmetric polynomial" (ESP) step-wise.
                            ESP = self.Base_Space().calO().Multiply_By(0)
                            ESP += sum([ self.__class__( ( S_VB , S_Mult*WCR(1) ) ) for S_VB , S_Mult in Vector_Bundle.Exterior_Power(Counter) ])
                            Factor_VB *= ESP

                        Factor_Mult = WCR(0)
                        for Degree2 , Coefficient2 in e.from_polynomial( Coefficient1 ).monomial_coefficients().items() :
                            #Factor2 += Coefficient2 * prod([ E[Counter-1] for Counter in Degree2 ])
                            Factor_Mult += Coefficient2 * prod([ Multiplicity.exterior_power(Counter) for Counter in Degree2 ])

                        alpha += Factor_VB.Multiply_By( Factor_Mult )

                    return alpha

            Result = self.Base_Space().calO().Multiply_By(0)
            for Partition_For_Summand in IntegerListsLex( n=p , length=len(self) ) :
                New_Summand = self.Base_Space().calO()
                New_Summand *= prod([ Subroutine_For_Exterior_Power_Of_Summand( Subexponent_For_Summand , self.Summands()[Summand_Counter] )
                                      for Summand_Counter , Subexponent_For_Summand in enumerate( Partition_For_Summand )
                                    ])
                Result += New_Summand
            return Result

        else : return self.Base_Space().calO().Multiply_By(0)

    # Synonyms for the function ``Exterior_Power``
    def Ext ( self , p ) :
        """Returns the ``e``-th exterior power of ``self``."""
        return self.Exterior_Power( p )
    def Alternating_Power ( self , p ) :
        """Returns the ``e``-th exterior power of ``self``."""
        return self.Exterior_Power( p )
    def Alt ( self , p ) :
        """Returns the ``e``-th exterior power of ``self``."""
        return self.Exterior_Power( p )
    def Wedge ( self , p ) :
        """Returns the ``e``-th exterior power of ``self``."""
        return self.Exterior_Power( p )


    def Global_Sections ( self ) :
        """Returns the global sections (i.e. the zero-th cohomology)."""
        return self.Cohomology(0)
    # Synonym for the function ``Global_Sections``
    def H0 ( self ) :
        """Returns the global sections (i.e. the zero-th cohomology)."""
        return self.Global_Sections()


    #def Is_Exceptional ( self , other ) : 


    def Is_SelfDual ( self ) :
        """Tests if `self` is self-dual."""
        return self == self.Dual()


    #def Is_SemiOrthogonal_To ( self , other ) :


    def Multiply_By ( self , External_Multiplicity ) :
        """Returns the tensor product of ``self`` with some multiplicity."""
        WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
        if External_Multiplicity in Integers() and 0 <= External_Multiplicity : External_Multiplicity = External_Multiplicity * WCR(1)
        assert type(External_Multiplicity) == type(WCR(0)) , 'The multiplicity need to be a Weyl character of ' + str(WCR) + '.'
        New = self.__class__( [ ( Summand , External_Multiplicity * Multiplicity ) for Summand , Multiplicity in self.Summands() ] )
        New.Tidy_Up()
        return New


    def Rank ( self ) :
        """Returns the rank of ``self``."""
        return sum([ Vector_Bundle.Rank() * Multiplicity.degree() for Vector_Bundle , Multiplicity in self.Summands() ])


    def Slope ( self ) :
        """Returns the slope of ``self``."""
        Degree = self.Degree()
        Rank = self.Rank()
        assert Degree in QQ , 'The ``Degree`` must be a rational number.'
        if Rank == 0 : return +Infinity
        else : return Degree / Rank


    def Standard_Stability ( self ) :
        """Returns the standard stability of ``self``."""
        Degree = self.Degree()
        Rank = self.Rank()
        assert Degree in QQ , 'The ``Degree`` must be a rational number.'
        return -1 * Degree + sqrt(-1) * Rank


    def Summands ( self , *Input ) :
        """Returns the attribute ``_Summands``."""
        if  len(Input) == 0 : return self._Summands
        elif len(Input) == 1 :
            Input = list(Input)[0]
            if Input in Integers() :
                Index = Input % len(self)
                return self.Summands()[Index]
            elif type(Input) == slice :
                Slice = Input
                return self.Summands()[Slice]
            else : raise ValueError('The input data is unknown.')
        else : raise ValueError('The input data is unknown.')


    def Tidy_Up ( self ) :
        """Returns ``self`` after merging summands with the same vector bundle and dropping zero summands."""
        WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
        Summands = []
        for Summand_Counter_Old , ( Vector_Bundle_Old , Multiplicity_Old ) in enumerate( self.Summands() ) :
            if Multiplicity_Old == 0 :
                pass
            else :
                Old_Summand_Is_Implemented_As_New_One = False
                for Summand_Counter_New , ( Vector_Bundle_New , Multiplicity_New ) in enumerate( Summands ) :
                    if Vector_Bundle_Old == Vector_Bundle_New :
                        Summands[Summand_Counter_New] = ( Vector_Bundle_Old , Multiplicity_Old+Multiplicity_New )
                        Old_Summand_Is_Implemented_As_New_One = True
                        break
                if not Old_Summand_Is_Implemented_As_New_One :
                    Summands += [ ( Vector_Bundle_Old , Multiplicity_Old ) ]

        if len( Summands ) == 0 :
            fw = self.Base_Space().Basis('fundamental weights')
            Summands = [ ( Irreducible_Equivariant_Vector_Bundle_Over_Irreducible_Homogeneous_Variety( Base_Space=self.Base_Space() , Highest_Weight=0*fw[1] ) , WCR(0) ) ]

        self._Summands = Summands


    #def Weights ( self ) :


    #def Symmetric_Power ( self , p ) :
    #    """Returns the ``e``-th symmetric power of ``self``."""
    #    assert p in Integers() , 'The exponent ``e`` must be an integer.'
    #
    #    def Symmetric_Power_Of_Summand ( Subexponent_For_Summand , Summand ) :
    #        Summand_VB , Summand_Mult = Summand
    #        S = self.Base_Space().calO().Multiply_By(0)
    #        for Partition_For_VB_And_Mult in IntegerListsLex( n=Subexponent_For_Summand , length=2 ) :
    #            Subexponent_For_VB , Subexponent_For_Mult = Partition_For_VB_And_Mult
    #            F2_Mult = Summand_Mult.symmetric_power(Subexponent_For_Mult)
    #            L = [ ( F1_VB , F1_Mult*F2_Mult ) for F1_VB , F1_Mult in Summand_VB.Symmetric_Power(Subexponent_For_VB) ]
    #            if len(L) == 0 : pass
    #            else : S += self.__class__( L )
    #        return S
    #
    #    Result = self.Base_Space().calO().Multiply_By(0)
    #    for Partition_For_Summand in IntegerListsLex( n=p , length=len(self) ) :
    #        New_Summand = self.Base_Space().calO()
    #        New_Summand *= prod([ Symmetric_Power_Of_Summand( Subexponent_For_Summand , self.Summands()[Summand_Counter] )
    #                              for Summand_Counter , Subexponent_For_Summand in enumerate( Partition_For_Summand )
    #                            ])
    #        Result += New_Summand
    #    return Result
    #
    ## Synonyms for the function ``Symmetric_Power``
    #def Sym ( self , p ) :
    #    """Returns the ``e``-th symmetric power of ``self``."""
    #    return self.Symmetric_Power( p )
    #def S ( self , p ) :
    #    """Returns the ``e``-th symmetric power of ``self``."""
    #     return self.Symmetric_Power( p )




In [2]:
G = Irreducible_Cartan_Group( 'B4' )
P = G.Parabolic_Subgroup( Excluded_Nodes={3} )
X = G/P
fw = X.Basis('fundamental weights')

#M = G.rmV( fw[1] )
Q = X.calU( 2*fw[4] )(-1) + X.calU().Dual()
print( Q.Det() )

#Rows = [ [ 'p' , 'S.Symmetric_Power(p)' , 'Rank' , 'binomial( S.Rank()+p-1 , p )' ] ]
#for p in [ -1 .. min([5,S.Rank()+1]) ] :
#    Result = S.Symmetric_Power(p)
#    Rows += [ [ p , Result , Result.Rank() , binomial( S.Rank()+p-1 , p ) ] ]
#if 5 < S.Rank()+1 : Rows += [ 4*[ '...' ] ]
#
#print( table( Rows , header_row=True ) )